In [1]:
# 必要ライブラリのインポート

%matplotlib inline
import numpy as np
import pandas as pd
import os
import re
import io
import requests
import matplotlib.pyplot as plt
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

In [2]:
# torch関連ライブラリのインポート
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable

In [3]:
import lightgbm as lgb
import xgboost as xgb
import catboost as catb

#from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

In [4]:
# デバイスの割り当て
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
data_new = pd.read_csv('data_new.csv')

In [6]:
data_new.columns

Index(['PassengerId', 'Perished', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'torch', 'Surname',
       'Ticket_id', 'Group_id', 'Title', 'WC_count', 'WCSurvived',
       'FamilySize', 'SmallFamily', 'LargeFamily', 'Deck', 'FirstName',
       'Ticket_freq', 'Pfare', 'Predict'],
      dtype='object')

In [7]:
data_new.head()

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,WC_count,WCSurvived,FamilySize,SmallFamily,LargeFamily,Deck,FirstName,Ticket_freq,Pfare,Predict
0,1,1.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,X,7.2500,...,NaN,NaN,2,1,0,N,Owen,1072,0.006763,NaN
1,2,0.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC,71.2833,...,1.0,NaN,2,1,0,C,Florence,92,0.774818,0.0
2,3,0.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,X,7.9250,...,1.0,NaN,1,0,0,N,Laina,1072,0.007393,NaN
3,4,0.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,X,53.1000,...,1.0,NaN,2,1,0,C,Lily,1072,0.049534,0.0
4,5,1.0,3,"Allen, Mr. William Henry",male,35.0,0,0,X,8.0500,...,NaN,NaN,1,0,0,N,William,1072,0.007509,NaN


In [8]:
data_new.loc[891:1308, 'Perished'] = data_new.loc[891:1308, 'Predict']

In [9]:
a = ['PassengerId','Name','Cabin','Age','Group_id','Predict','Surname','Ticket_id','Title','WC_count','WCSurvived']

In [10]:
data_new = data_new.drop(columns=a)

In [11]:
data_new.head()

,Perished,Pclass,Sex,SibSp,Parch,Ticket,Fare,Embarked,torch,FamilySize,SmallFamily,LargeFamily,Deck,FirstName,Ticket_freq,Pfare
0,1.0,3,male,1,0,X,7.2500,S,NaN,2,1,0,N,Owen,1072,0.006763
1,0.0,1,female,1,0,PC,71.2833,C,NaN,2,1,0,C,Florence,92,0.774818
2,0.0,3,female,0,0,X,7.9250,S,NaN,1,0,0,N,Laina,1072,0.007393
3,0.0,1,female,1,0,X,53.1000,S,NaN,2,1,0,C,Lily,1072,0.049534
4,1.0,3,male,0,0,X,8.0500,S,NaN,1,0,0,N,William,1072,0.007509


In [12]:
def dummies(dataset, i, ):
    dummies = pd.get_dummies(dataset[i])
    dataset = pd.concat([ dataset, dummies], axis=1)
    dataset = dataset.drop(i, axis=1)
    return dataset

In [13]:
def dumm_SS(dataset, category, numerical_cols):
    dataset = pd.get_dummies(dataset, columns=category)
    SS = StandardScaler()
    for i in numerical_cols:
        dataset[i] = SS.fit_transform(dataset[i].values.reshape(-1, 1))
    return dataset

In [14]:
category = ['Pclass','Ticket','Embarked','Sex','Deck','FirstName']
numerical_cols = ['Fare','Ticket_freq','Pfare']

In [15]:
data_new.columns

Index(['Perished', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Embarked', 'torch', 'FamilySize', 'SmallFamily', 'LargeFamily', 'Deck',
       'FirstName', 'Ticket_freq', 'Pfare'],
      dtype='object')

In [16]:
train = data_new[['Pclass', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Embarked', 'FamilySize', 'SmallFamily', 'LargeFamily', 'Deck',
       'Ticket_freq', 'Pfare','FirstName']]
Y_train= data_new['Perished'].values
train = dumm_SS(train,category,numerical_cols)
train = np.array(train)

In [17]:
# 損失計算用
def eval_loss(loader, device, net, criterion):
  
    # データローダーから最初の1セットを取得する
    for images, labels in loader:
        break

    # デバイスの割り当て
    inputs = images.to(device).float()
    labels = labels.to(device).long()

    # 予測計算
    outputs = net(inputs.float()).float()

    #  損失計算
    loss = criterion(outputs.float(), labels.long())

    return loss

In [18]:
# 学習用関数
def fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history):

    # tqdmライブラリのインポート
    from tqdm.notebook import tqdm

    base_epochs = len(history)
  
    for epoch in range(base_epochs, num_epochs+base_epochs):
        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0

        #訓練フェーズ
        net.train()
        count = 0

        for inputs, labels in tqdm(train_loader,disable=True):
            count += len(labels)
            inputs = inputs.to(device).float()
            labels = labels.to(device).long()

            # 勾配の初期化
            optimizer.zero_grad()

            # 予測計算
            outputs = net(inputs.float()).float()
            
            # 損失計算
            loss = criterion(outputs, labels)
            train_loss += loss.item()

            # 勾配計算
            loss.backward()

            # パラメータ修正
            optimizer.step()

            # 予測値算出
            predicted = torch.max(outputs, 1)[1]

            # 正解件数算出
            train_acc += (predicted == labels).sum().item()

            # 損失と精度の計算
            avg_train_loss = train_loss / count
            avg_train_acc = train_acc / count

        #予測フェーズ
        net.eval()
        count = 0

        for inputs, labels in test_loader:
            count += len(labels)
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 予測計算
            outputs = net(inputs)

            # 損失計算
            loss = criterion(outputs, labels.long())
            val_loss += loss.item()

            # 予測値算出
            predicted = torch.max(outputs, 1)[1]

            # 正解件数算出
            val_acc += (predicted == labels).sum().item()

            # 損失と精度の計算
            avg_val_loss = val_loss / count
            avg_val_acc = val_acc / count
    
        #print (f'Epoch [{(epoch+1)}/{num_epochs+base_epochs}], loss: {avg_train_loss:.5f} acc: {avg_train_acc:.5f} val_loss: {avg_val_loss:.5f}, val_acc: {avg_val_acc:.5f}')
        item = np.array([epoch+1, avg_train_loss, avg_train_acc, avg_val_loss, avg_val_acc])
        history = np.vstack((history, item))
    return history

In [19]:
# 学習ログ解析

def evaluate_history(history):
    #損失と精度の確認
    print(f'初期状態: 損失: {history[0,3]:.5f} 精度: {history[0,4]:.5f}') 
    print(f'最終状態: 損失: {history[-1,3]:.5f} 精度: {history[-1,4]:.5f}' )
    print(lr)
    # Evaluate the model
    test_var = Variable(torch.FloatTensor(x_test), requires_grad=True)
    with torch.no_grad():
        result = net(test_var)
    values, labels = torch.max(result, 1)
    num_right = np.sum(labels.data.numpy() == y_test)
    print('Accuracy {:.2f}'.format(num_right / len(y_test)))
    
    num_epochs = len(history)
    unit = num_epochs / 10

    # 学習曲線の表示 (損失)
    fig = plt.figure(figsize=(9,5))
    ax = fig.add_subplot(121)
    ax.plot(history[:,0], history[:,1], 'b', label='train')
    ax.plot(history[:,0], history[:,3], 'k', label='test')
    #ax.xticks(np.arange(0,num_epochs+1, unit))
    ax.set_xlabel('epoch')
    ax.set_ylabel('loss')
    #ax.title('学習曲線(損失)')
    ax.legend()

    # 学習曲線の表示 (精度)
    ax = fig.add_subplot(122)
    ax.plot(history[:,0], history[:,2], 'b', label='train')
    ax.plot(history[:,0], history[:,4], 'k', label='accracy')
    #ax.xticks(np.arange(0,num_epochs+1,unit))
    ax.set_xlabel('epoch')
    ax.set_ylabel('accracy')
    #ax.title('学習曲線(精度)')
    ax.legend()
    
    plt.show()

In [20]:
# PyTorch乱数固定用
def torch_seed(seed=123):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True

In [21]:
train.shape

(1309, 603)

In [22]:
%%time
# データローダーの定義

# ミニバッチのサイズ指定
batch_size = 32

# 訓練用データローダー
x_train, x_test, y_train, y_test = train_test_split(train, Y_train, test_size = 0.1, random_state=90)

# 訓練用なので、シャッフルをかける
train_data = TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(y_train).float())
train_loader1 = DataLoader(train_data, batch_size = batch_size, shuffle=True)

# 検証用データローダー
# 検証時にシャッフルは不要
test_data = TensorDataset(torch.from_numpy(x_test).float(), torch.from_numpy(y_test).float())
test_loader1 = DataLoader(train_data, batch_size, shuffle=False)

class CNN(nn.Module):
      def __init__(self):
        super().__init__()
        self.relu = nn.ReLU(inplace = True)
        self.l1 = nn.Linear(603,1024)
        self.l2 = nn.Linear(1024,1024)
        self.l3 = nn.Linear(1024,512)
        self.l4 = nn.Linear(512,2)
        self.dropput1 = nn.Dropout(0.4)
        self.dropput2 = nn.Dropout(0.5)

        self.classifier = nn.Sequential(
            self.l1,
            self.relu,
            self.l2,
            self.relu,
            self.l3,
            self.relu,
            self.l4
        )

      def forward(self, x):
        x = self.classifier(x)
        return x      

# モデルインスタンス生成
net = CNN().to(device)

# 損失関数： 交差エントロピー関数
criterion = nn.CrossEntropyLoss()

# 学習率
lr = 0.001

# 最適化関数: 勾配降下法
optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0)

# モデルの概要表示
print(net)

# 損失計算
loss = eval_loss(test_loader1, device, net, criterion)



# 乱数初期化
torch_seed()

# 繰り返し回数
num_epochs = 15000

# 評価結果記録用
history = np.zeros((0,5))

# 学習
history = fit(net, optimizer, criterion, num_epochs, train_loader1, test_loader1, device, history)

evaluate_history(history)

CNN(
  (relu): ReLU(inplace=True)
  (l1): Linear(in_features=603, out_features=1024, bias=True)
  (l2): Linear(in_features=1024, out_features=1024, bias=True)
  (l3): Linear(in_features=1024, out_features=512, bias=True)
  (l4): Linear(in_features=512, out_features=2, bias=True)
  (dropput1): Dropout(p=0.4, inplace=False)
  (dropput2): Dropout(p=0.5, inplace=False)
  (classifier): Sequential(
    (0): Linear(in_features=603, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=512, out_features=2, bias=True)
  )
)


KeyboardInterrupt: 

In [23]:
train[891:1309].shape

(418, 603)

In [24]:
data_new = pd.read_csv('data_new.csv')
test = data_new[['Pclass', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Embarked', 'FamilySize', 'SmallFamily', 'LargeFamily', 'Deck',
       'Ticket_freq', 'Pfare','FirstName']]
test = test.loc[891:1308]
test = dumm_SS(test,category,numerical_cols)
test = np.array(test)

In [25]:
test = train[891:1309]

In [26]:
#予測フェーズ
with torch.no_grad():
    result = net(torch.FloatTensor(test))
values, labels = torch.max(result, 1)
result = labels.numpy()

In [27]:
data_new.loc[891:1308,'Predict'] = result
data = data_new

In [28]:
# これから先が難しい，よう変更
# データが少ない過学習に陥る、femaleの予測が難しい　予測しようとすると全て同じ値を返してしまう
# 残ったデータでは男は1が多くて→機会学習で予測はいい感じ
# 女は0が多い予想が難しい。データの少なさに起因。なのでなるべく手動で分類したい　80%以上は全て手動にする？？
#女は１を予想するのが難しい
# いい分類方法がわからない
#いいクラス,DeckがNではないものは0にする→これをしてしまうとデータが少なすぎて予測ができない
#以下の処理をしてもあまり精度は向上しない
#以下のものは予測ができていてやはりそれ以外のものを外してる
#ageが欠損してると死んでる可能性が上がる
data['Predict'][(data.Age.isnull())&(data.Sex == 'female')&(data.Ticket!='X')] = 1
data['Predict'][(data['WCSurvived'].isnull()) & (data.Sex == 'female') & (data.Deck != 'N') ] = 0
data['Predict'][(data['WCSurvived'].isnull()) & (data.Sex == 'female') & (data.Pclass == 1) ] = 0
data['Predict'][(data.Sex == 'female') & (data.Pclass ==1)] = 0
#WC_count>=2だと生きている
data['Predict'][(data.Sex == 'female') & (data.Pclass ==2)&(data.Deck == 'F')] = 0

In [29]:
result = data.loc[891:1308,'Predict']

In [30]:
df = pd.DataFrame(
    { 'torch':result }
)

In [31]:
df.to_csv('torch.csv',index = False)

In [37]:
data['FirstName'].value_counts()

William    48
John       39
Charles    23
Mary       22
George     21
           ..
Janko       1
Anne        1
Stefo       1
Satio       1
Woolf       1
Name: FirstName, Length: 570, dtype: int64

In [38]:
data[data['FirstName']=='William']

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,WC_count,WCSurvived,FamilySize,SmallFamily,LargeFamily,Deck,FirstName,Ticket_freq,Pfare,Predict
4,5,1.0,3,"Allen, Mr. William Henry",male,35.0,0,0,X,8.0500,...,NaN,NaN,1,0,0,N,William,1072,0.007509,NaN
12,13,1.0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,X,8.0500,...,NaN,NaN,1,0,0,N,William,1072,0.007509,NaN
23,24,0.0,1,"Sloper, Mr. William Thompson",male,28.0,0,0,X,35.5000,...,NaN,NaN,1,0,0,A,William,1072,0.033116,NaN
45,46,1.0,3,"Rogers, Mr. William John",male,NaN,0,0,X,8.0500,...,NaN,NaN,1,0,0,N,William,1072,0.007509,NaN
59,60,1.0,3,"Goodwin, Master. William Frederick",male,11.0,5,2,CA,46.9000,...,6.0,1.0,8,0,1,N,William,68,0.689706,NaN
86,87,1.0,3,"Ford, Mr. William Neal",male,16.0,1,3,WC,34.3750,...,NaN,NaN,5,0,1,N,William,15,2.291667,NaN
97,98,0.0,1,"Greenfield, Mr. William Bertram",male,23.0,0,1,PC,63.3583,...,NaN,NaN,2,1,0,D,William,92,0.688677,NaN
117,118,1.0,2,"Turpin, Mr. William John Robert",male,29.0,1,0,X,21.0000,...,NaN,NaN,2,1,0,N,William,1072,0.019590,NaN
191,192,1.0,2,"Carbines, Mr. William",male,19.0,0,0,X,13.0000,...,NaN,NaN,1,0,0,N,William,1072,0.012127,NaN
226,227,0.0,2,"Mellors, Mr. William John",male,19.0,0,0,X,10.5000,...,NaN,NaN,1,0,0,N,William,1072,0.009795,NaN
